In [1]:
import pandas as pd

In [10]:
ticker = "NFLX"

In [14]:
def read_annual_data(csv_file, ticker):
    data = pd.read_csv(csv_file, index_col=0)
    data = data[ticker]
    data = data.pct_change()
    data = data.loc[2009:2019]
    return data

In [47]:
def read_quartely_data(csv_file, ticker):
    data = pd.read_csv(csv_file, index_col=0, parse_dates=True)
    data = data[ticker]
    data = data.resample('Y', convention='end').agg('mean')
    data = data.pct_change()
    data = data.loc['2009-01-01':'2019-12-31']
    data.index = data.index.year
    return data

In [51]:
def read_quartely_ratio(csv_file, ticker):
    data = pd.read_csv(csv_file, index_col=0, parse_dates=True)
    data = data[ticker]
    data = data.resample('Y', convention='end').agg('mean')
    data = data.loc['2009-01-01':'2019-12-31']
    data.index = data.index.year
    return data

In [20]:
rev = read_annual_data("50-rev.csv", ticker)
eps = read_annual_data("50-eps.csv", ticker)
fcf = read_annual_data("50-fcf.csv", ticker)

In [48]:
bvps = read_quartely_data("50-bvps.csv", ticker)

In [52]:
roi = read_quartely_ratio("50-roi.csv", ticker)
de = read_quartely_ratio("50-de-ratio.csv", ticker)
pe = read_quartely_ratio("50-pe-ratio.csv", ticker)
current = read_quartely_ratio("50-current-ratio.csv", ticker)

In [21]:
def add_annual_data(writer, df, sheet_name):
    df.to_excel(writer, sheet_name=sheet_name)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Create a new chart object.
    chart = workbook.add_chart({'type': 'line'})

    # Add a series to the chart.
    chart.add_series({
        'name': sheet_name,
        'categories': [sheet_name, 1, 0, 11, 0],
        'values': [sheet_name, 1, 1, 11, 1],
    })
    
    # To set the labels on x axis not on 0 and rotate
    chart.set_x_axis({
        'label_position': 'low',
        'num_font':  {'rotation': 45}
    })


    # Insert the chart into the worksheet.
    worksheet.insert_chart('D1', chart)

In [54]:
writer = pd.ExcelWriter("data3.xlsx", engine='xlsxwriter')

add_annual_data(writer, rev, "Revenue")
add_annual_data(writer, eps, "EPS")
add_annual_data(writer, fcf, "FCF")
add_annual_data(writer, bvps, "BVPS")
add_annual_data(writer, roi, "ROI")
add_annual_data(writer, de, "DE")
add_annual_data(writer, pe, "PE")
add_annual_data(writer, current, "Current")

writer.close()